In [745]:
import json
import pennylane as qml
import pennylane.numpy as np




# Put any helper functions here that you want to make #

def rotation_and_controls(k, bits):
    control_values = [int(b) for b in f"{k:0{bits}b}"]
    return 2 * np.arcsin(1 / k), control_values


def encode_hermitian(A, wires):
    """
    Encodes a hermitian matrix A as a unitary U = e^{iA}.

    Args
        - A (numpy.tensor): a 2x2 matrix
        - b (numpy.tensor): a length-2 vector

    Returns
        - (qml.Operation): a unitary operation U = e^{iA}
    """
    return qml.exp(qml.Hermitian(A, wires=wires), coeff=1j)


def mint_to_lime(A, b):
    """
    Calculates the optimal mint and lime proportions in the Mojito HHLime twist.

    Args
        - A (numpy.tensor): a 2x2 matrix
        - b (numpy.tensor): a length-2 vector

    Returns
        - x (numpy.tensor): the solution to Ax = b
        (int): the number of operations in your HHL circuit.
    """
    b_qubits = 1
    b_wires = [0]

    qpe_qubits = 10
    qpe_wires = list(range(b_qubits, b_qubits + qpe_qubits))

    ancilla_qubits = 150
    ancilla_wires = list(
        range(b_qubits + qpe_qubits, ancilla_qubits + b_qubits + qpe_qubits)
    )

    all_wires = b_wires + qpe_wires + ancilla_wires
    dev = qml.device("default.qubit", wires=all_wires)

    @qml.qnode(dev)
    def HHL(A, b):
        """
        Implements the HHL algorithm.
        Args
            - A (numpy.tensor): a 2x2 matrix
            - b (numpy.tensor): a length-2 vector

        Returns
            - (numpy.tensor):
                The probability distribution for the vector x, which is the
                solution to Ax = b.
        """


        # Put your code here #
        qml.StatePrep(b, wires=b_wires)
        U = encode_hermitian(A, wires=b_wires)
        qpe_to_use = 10
        qml.QuantumPhaseEstimation(
            U,
            estimation_wires=qpe_wires[:qpe_to_use]
        )
        for k in range(1, 2 ** qpe_to_use):
            phi, control_values = rotation_and_controls(k, qpe_to_use)
            qml.ctrl(qml.RY(phi, wires=ancilla_wires[0]), control=qpe_wires[:qpe_to_use], control_values=control_values)
        qml.measure(wires=ancilla_wires[0], postselect=1)
        qml.adjoint(qml.QuantumPhaseEstimation(
            U,
            estimation_wires=qpe_wires[:qpe_to_use]
        ))
        return qml.probs(wires=b_wires)

    # we return probs, but we need the state itself (it will be real-valued)
    x = np.sqrt(HHL(A, b))
    #print(x)
    #print(HHL.tape._ops)
    return x, len(HHL.tape._ops)



In [744]:
mint_to_lime(np.array([[1, -0.333333], [-0.333333, 1]]), np.array([0.48063554, 0.87692045]))


(tensor([0.6015496 , 0.79883545], requires_grad=True), 259)

In [348]:
t = 3 * np.pi / 4
2 * np.arcsin(1/(4*(2/3)*t/(2*np.pi)))

3.141592653589793

In [660]:
[1,2,3][::2]

[1, 3]

In [35]:
mydev = qml.device("default.qubit", wires=2)

@qml.qnode(mydev)
def my_circuit():
    qml.PauliX(0)
    # qml.CRY(np.pi, wires=[1, 0])
    return qml.probs()

In [36]:
my_circuit()

tensor([0., 0., 1., 0.], requires_grad=True)

In [56]:
np.pi / (2*np.arcsin(1/4))

6.216537678860817

<function pennylane.math.multi_dispatch.expm(tensor, like=None)>

In [747]:


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    A, b = json.loads(test_case_input)
    output, num_ops = mint_to_lime(np.array(A), np.array(b))
    output = output.tolist()
    output.append(num_ops)
    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    output = solution_output[:-1]
    num_ops = solution_output[-1]
    expected_output = json.loads(expected_output)

    assert num_ops > 4, "Your circuit should have a few more operations!"
    print(f"Solution output: {output}; Expected output: {expected_output}")
    assert np.allclose(output, expected_output, rtol=1e-2)


# These are the public test cases
test_cases = [
    ('[[[1, -0.333333], [-0.333333, 1]], [0.48063554, 0.87692045]]', '[0.6123100731658992, 0.7906177169127275]'),
    ('[[[0.456, -0.123], [-0.123, 0.123]], [0.96549299, 0.26042903]]', '[0.5090526763759141, 0.8607353673888718]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[[1, -0.333333], [-0.333333, 1]], [0.48063554, 0.87692045]]'...


Solution output: [0.6123100731658556, 0.7906177169126805]; Expected output: [0.6123100731658992, 0.7906177169127275]
Correct!
Running test case 1 with input '[[[0.456, -0.123], [-0.123, 0.123]], [0.96549299, 0.26042903]]'...
Solution output: [0.5090526763759164, 0.8607353673888463]; Expected output: [0.5090526763759141, 0.8607353673888718]
Correct!
